In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import country_converter as coco 

# Import API key
from api_key import gkey

gmaps.configure(api_key=gkey)

In [2]:
# load csv
# read from new csv
file_path = "../output_data/cityweather.csv"
city_weather_gmaps = pd.read_csv(file_path)

city_weather_gmaps.head()


,retrieval index,City,Country Code,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed(mph),Date
0,"kristiansund,no",Kristiansund,NO,63.1115,7.7320,55.45,87.0,75.0,10.36,2022-06-23 01:13:27
1,"pisco,pe",Pisco,PE,-13.7000,-76.2167,62.65,77.0,100.0,10.36,2022-06-23 01:13:27
2,"port-gentil,ga",Port-Gentil,GA,-0.7193,8.7815,72.09,79.0,100.0,12.84,2022-06-23 01:13:27
3,"kawalu,id",Kawalu,ID,-7.3817,108.2082,75.42,90.0,99.0,0.85,2022-06-23 01:13:27
4,"jipijapa,ec",Jipijapa,EC,-1.3333,-80.5833,67.50,86.0,99.0,5.55,2022-06-23 01:13:28


In [3]:
# create humidity heatmap
city_coordinates = city_weather_gmaps[["Latitude", "Longitude"]]
humidity_level = city_weather_gmaps["Humidity (%)"]



In [17]:
# create dataframe fitting weather criteria
ideal_weather_cities = city_weather_gmaps.loc[(city_weather_gmaps['Max Temperature (F)'] >=80) & 
                                            (city_weather_gmaps['Max Temperature (F)'] < 85) & 
                                            (city_weather_gmaps['Humidity (%)'] <= 67) &
                                            (city_weather_gmaps['Humidity (%)'] > 40) &
                                            (city_weather_gmaps['Cloudiness (%)'] > 10) &
                                            (city_weather_gmaps['Cloudiness (%)'] > 15)]
    
ideal_weather_cities.head()

,retrieval index,City,Country Code,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed(mph),Date
53,"poso,id",Poso,ID,-1.3959,120.7524,83.32,67.0,89.0,2.01,2022-06-23 01:13:42
59,"houma,cn",Houma,CN,35.6254,111.3627,81.57,64.0,23.0,3.29,2022-06-23 01:13:44
144,"faanui,pf",Faanui,PF,-16.4833,-151.7500,80.33,66.0,20.0,8.79,2022-06-23 01:14:10
150,"chengalpattu,in",Chengalpattu,IN,12.7000,79.9833,83.91,66.0,100.0,5.44,2022-06-23 01:09:20
199,"lorengau,pg",Lorengau,PG,-2.0226,147.2712,84.69,67.0,38.0,5.10,2022-06-23 01:14:25


In [10]:
# create new dataframe to store hotel names and country located

hotel_df = ideal_weather_cities.loc[:, ["Latitude", "Longitude", "City", "Country Code"]].reset_index(drop=True)
hotel_df["Country"] = hotel_df['Country Code'].apply(lambda x: coco.convert(names=x, to='name_short'))
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Latitude,Longitude,City,Country Code,Country,Hotel Name
0,-1.3959,120.7524,Poso,ID,Indonesia,
1,35.6254,111.3627,Houma,CN,China,
2,-16.4833,-151.7500,Faanui,PF,French Polynesia,
3,12.7000,79.9833,Chengalpattu,IN,India,
4,-2.0226,147.2712,Lorengau,PG,Papua New Guinea,


In [11]:
# use searchnearby to obtain hotel names
        # store hotel names to dataframe

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey
}

for index, row in hotel_df.iterrows():
        lat = row["Latitude"]
        lng = row["Longitude"]
        params["location"] = f"{lat},{lng}"
        nearby_search_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        hotel_name = requests.get(nearby_search_url, params=params)
        hotel_json = hotel_name.json()
        try:
                hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][0]["name"]
        except (KeyError, IndexError):
                pass

In [18]:
hotel_df.head()

,Latitude,Longitude,City,Country Code,Country,Hotel Name
0,-1.3959,120.7524,Poso,ID,Indonesia,Ancyra Hotel By Continent - Poso
1,35.6254,111.3627,Houma,CN,China,Atour Hotel
2,-16.4833,-151.7500,Faanui,PF,French Polynesia,Four Seasons Resort Bora Bora
3,12.7000,79.9833,Chengalpattu,IN,India,Fairfield by Marriott Chennai Mahindra World City
4,-2.0226,147.2712,Lorengau,PG,Papua New Guinea,Lorengau Harbourside Hotel


In [16]:
# configure gmap
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout, map_type='HYBRID')

# add heat layer
heatmap_layer = gmaps.heatmap_layer(city_coordinates, weights=humidity_level, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
fig.add_layer(heatmap_layer)

# add marker layer
hotel_coordinates = hotel_df[['Latitude', 'Longitude']]

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()] 

markers = gmaps.marker_layer(hotel_coordinates, info_box_content=hotel_info)

fig.add_layer(markers)

fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…